<a href="https://colab.research.google.com/github/Himanshu-jn52/-poem-or-novel-Classifier-AI-ML/blob/main/poemVSnovel.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing required libraries.

In [95]:
import re
import nltk
import pickle
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import GaussianNB,MultinomialNB
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

# **DATA PREPROCESSING**

Collecting required infromation from dataset provided and adding to the dataframe.
Dataframe has two columns:
1.   Content - contents of Poem and Novel
2.   Index - for Poem = 0 and for Novel = 1



In [96]:
df = pd.DataFrame()
df1 = pd.read_csv("Dataset/all.csv",usecols=["author", "content", "poem name", "age","type"])
df2 = pd.read_csv("Dataset/booksummaries600.txt",delimiter = "\t",header=None)
df2 = df2.drop([0,1], axis = 1)
df2.columns = ["novel name", "author", "date", "type", "content"]

for x in df1["content"][:500]:
    try:
      df = df.append([[x, 0]],ignore_index=True)
    except:
      continue

for x in df2["content"][0:500]:
    try:
      df = df.append([[x, 1]],ignore_index=True)
    except:
      continue

df.columns = ['Content', 'Index']

Cleaning the data 

In [97]:
cleaned_data = []             #to store cleaned data
for i in range(0, 1000):
  content = re.sub('[^a-zA-Z]', ' ', df['Content'][i])    #to replace all character & punctuation with white space
  content = content.lower()        
  content = content.split()
  ps = PorterStemmer()   # Creating an stem object
  all_stopwords = stopwords.words('english')   # collecting all the stopwords in english
  content = [ps.stem(word) for word in content if not word in set(all_stopwords)] # removing stopwords from contents
  content = ' '.join(content)
  cleaned_data.append(content)

Converting contents into binary format (Creating BOW)

In [98]:
cv = CountVectorizer(max_features = 1500)
X = cv.fit_transform(cleaned_data).toarray() #fit and transform to vector
y = df.iloc[:, -1].values


# **MODELLING & TRAINING**

Generating train & test data with test size of 20%.

In [99]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 0)

Defining two methods of Naive - Bayes

1.  Gaussian Naive Bayes
2.  Multinomial Naive Bayes




In [100]:
#Gaussian Naive Bayes
Gclassifier = GaussianNB() 
Gclassifier.fit(X_train, y_train)

#Multinomial Naive Bayes
Mclassifier = MultinomialNB()
Mclassifier.fit(X_train, y_train)

print(Gclassifier.score(X_test,y_test)) #---> 0.905
print(Mclassifier.score(X_test,y_test)) #---> 0.98 will proceed with Multinomial Naive Bayes

0.905
0.98


Performing prediction for Multinomial Naive Bayes 

In [101]:
y_pred = Mclassifier.predict(X_test)
np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1)

array([[1, 1],
       [1, 1],
       [0, 0],
       [1, 1],
       [1, 1],
       [1, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 1],
       [0, 0],
       [1, 1],
       [1, 1],
       [0, 0],
       [1, 1],
       [0, 0],
       [1, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 1],
       [1, 1],
       [0, 0],
       [0, 0],
       [1, 1],
       [0, 0],
       [1, 1],
       [1, 1],
       [1, 1],
       [1, 1],
       [0, 0],
       [1, 1],
       [1, 1],
       [0, 0],
       [1, 1],
       [0, 0],
       [0, 0],
       [1, 1],
       [1, 1],
       [1, 1],
       [0, 0],
       [1, 1],
       [1, 1],
       [0, 0],
       [1, 0],
       [1, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [1, 1],
       [1, 1],
       [0, 0],
       [0, 0],
       [1, 1],
       [0, 0],
       [1, 1],
       [1, 1],
       [0, 0],
       [0, 0],
       [0, 0],
       [0, 0],
       [0,

Will evaluate the modela and check the accuracy score.

In [102]:
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[ 94   4]
 [  0 102]]


0.98

In [103]:

# dump trained model to pickle file
pickle.dump(Mclassifier, open("model.pkl", "wb"))

In [111]:
python "/content/Main.py"

SyntaxError: ignored